In [9]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
from datasets import load_dataset

dataset = load_dataset("mediabiasgroup/BABE")
print(dataset)



DatasetDict({
    train: Dataset({
        features: ['text', 'outlet', 'label', 'topic', 'news_link', 'biased_words', 'uuid', 'type', 'label_opinion'],
        num_rows: 3121
    })
    test: Dataset({
        features: ['text', 'outlet', 'label', 'topic', 'news_link', 'biased_words', 'uuid', 'type', 'label_opinion'],
        num_rows: 1000
    })
})


In [11]:
!pip install transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
!pip install -U 'transformers[torch]' accelerate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_dataset, DatasetDict

# Step 1: Load the dataset
dataset = load_dataset("mediabiasgroup/BABE")

# Step 2: Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Step 3: Define a tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Step 4: Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Step 5: Split the dataset into train/validation (10% of training set)
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1, seed=42)

# Step 6: Recreate the dataset with train, validation, and test splits
tokenized_datasets = DatasetDict({
    "train": train_test_split["train"],
    "validation": train_test_split["test"],
    "test": tokenized_datasets["test"]
})

# Step 7: Define the model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Step 8: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Step 9: Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Step 10: Train the model
trainer.train()

# Step 11: Evaluate the model on the test set
results = trainer.evaluate(tokenized_datasets["test"])
print("Test set results:", results)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.346871
2,No log,0.387127
3,0.302800,0.486211


Test set results: {'eval_loss': 0.38078516721725464, 'eval_runtime': 32.511, 'eval_samples_per_second': 30.759, 'eval_steps_per_second': 1.938, 'epoch': 3.0}


In [26]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

example_news = '''Finally, taxpayer money is being redirected away from these left-wing indoctrination centers.

It is encouraging to see that Trump is not just targeting Ivy League schools but extending this crackdown to universities across the board.
'''
example_text = example_news
prediction = classifier(example_text)
print("Prediction: 1: Biased, 0: Biased", prediction)



Device set to use mps:0


Prediction: 1: Biased, 0: Biased [{'label': 'LABEL_1', 'score': 0.9829651713371277}]
